<a href="https://colab.research.google.com/github/Luqman0033/Assignment3/blob/main/ir_system_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install nltk scikit-learn rank-bm25 numpy
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [26]:
from google.colab import files
import pandas as pd

print("Upload your dataset CSV file (must contain doc_id and text columns)")

uploaded = files.upload()

csv_filename = list(uploaded.keys())[0]
print("Uploaded file:", csv_filename)

try:
    df = pd.read_csv(csv_filename)
except:
    df = pd.read_csv(csv_filename, encoding='latin1')

print("Loaded dataset with shape:", df.shape)
df.head()



Upload your dataset CSV file (must contain doc_id and text columns)


Saving Articles.csv to Articles (2).csv
Uploaded file: Articles (2).csv
Loaded dataset with shape: (2692, 4)


,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [27]:

text_candidates = ["text", "Text", "article", "Article", "ArticleText", "Article_text",
                   "Heading", "heading", "Body", "body", "content", "Content", "News"]

id_candidates = ["doc_id", "id", "ID", "article_id", "ArticleID", "filename"]

found_text_col = None
for c in text_candidates:
    if c in df.columns:
        found_text_col = c
        break

if found_text_col is None:
    raise Exception("Could not find a suitable text column in the CSV. "
                    "Available columns: {}".format(list(df.columns)))

if found_text_col != "text":
    df = df.rename(columns={found_text_col: "text"})
    print(f"Renamed column '{found_text_col}' -> 'text'")

found_id_col = None
for c in id_candidates:
    if c in df.columns:
        found_id_col = c
        break

if found_id_col:
    if found_id_col != "doc_id":
        df = df.rename(columns={found_id_col: "doc_id"})
        print(f"Renamed column '{found_id_col}' -> 'doc_id'")
else:

    df = df.reset_index(drop=True)
    df["doc_id"] = df.index.to_series().apply(lambda i: f"doc_{i+1}")
    print("Created 'doc_id' column from DataFrame index (doc_1, doc_2, ...).")


assert "doc_id" in df.columns and "text" in df.columns, \
       "After normalization, df must contain 'doc_id' and 'text' columns."

print("\nPreview (doc_id + first 200 characters of text):")
preview = df[["doc_id", "text"]].head(10).copy()
preview["text_snippet"] = preview["text"].astype(str).str[:200].str.replace("\n", " ")
print(preview[["doc_id", "text_snippet"]].to_string(index=False))


doc_ids = df["doc_id"].astype(str).tolist()
docs = df["text"].astype(str).tolist()

print("\nTotal documents prepared for indexing:", len(docs))


Renamed column 'Article' -> 'text'
Created 'doc_id' column from DataFrame index (doc_1, doc_2, ...).

Preview (doc_id + first 200 characters of text):
doc_id                                                                                                                                                                                             text_snippet
 doc_1 KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources s
 doc_2 HONG KONG: Asian markets started 2015 on an upswing in limited trading on Friday, with mainland Chinese stocks surging in Hong Kong on speculation Beijing may ease monetary policy to boost slowing gro
 doc_3 HONG KONG:  Hong Kong shares opened 0.66 percent lower Monday following a tepid lead from Wall Street, as the first full week of the new year kicked off.The benchmark Hang Seng Index dipped 158.63 poi
 doc_4 HONG KONG:

In [28]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

class Preprocessor:
    def __init__(self, do_stopwords=True, do_stemming=False):
        self.stopwords = set(stopwords.words("english")) if do_stopwords else set()
        self.stemmer = PorterStemmer() if do_stemming else None

    def tokenize(self, text):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t.isalnum()]
        if self.stopwords:
            tokens = [t for t in tokens if t not in self.stopwords]
        if self.stemmer:
            tokens = [self.stemmer.stem(t) for t in tokens]
        return tokens

    def preprocess(self, text):
        return " ".join(self.tokenize(text))

pre = Preprocessor(do_stopwords=True, do_stemming=False)


In [29]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer


tokenized_docs = [pre.tokenize(d) for d in docs]
processed_docs = [" ".join(tokens) for tokens in tokenized_docs]

bm25 = BM25Okapi(tokenized_docs)
print("BM25 index built.")

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)
print("TF-IDF matrix shape:", tfidf_matrix.shape)


BM25 index built.
TF-IDF matrix shape: (2692, 25107)


In [30]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def search_bm25(query, top_k=10):
    q_tokens = pre.tokenize(query)
    scores = bm25.get_scores(q_tokens)
    top_idx = np.argsort(scores)[::-1][:top_k]
    return [(doc_ids[i], float(scores[i]), i) for i in top_idx]

def search_tfidf(query, top_k=10):
    q_vec = tfidf_vectorizer.transform([pre.preprocess(query)])
    sims = cosine_similarity(q_vec, tfidf_matrix).flatten()
    top_idx = np.argsort(sims)[::-1][:top_k]
    return [(doc_ids[i], float(sims[i]), i) for i in top_idx]

def search_bm25_tfidf(query, bm25_k=30, final_k=10):
    q_tokens = pre.tokenize(query)
    bm25_scores = bm25.get_scores(q_tokens)
    cand_idx = np.argsort(bm25_scores)[::-1][:bm25_k]

    q_vec = tfidf_vectorizer.transform([pre.preprocess(query)])
    cand_matrix = tfidf_matrix[cand_idx]

    sims = cosine_similarity(q_vec, cand_matrix).flatten()
    order = np.argsort(sims)[::-1][:final_k]

    results = []
    for r in order:
        idx = cand_idx[r]
        results.append((doc_ids[idx], float(sims[r]), idx))
    return results

print("Retrieval functions ready.")


Retrieval functions ready.


In [31]:
query = input("Enter your search query: ")

print("\n BM25 RESULTS:")
for rank, (docid, score, idx) in enumerate(search_bm25(query, 5), start=1):
    print(f"{rank}. {docid}  |  score={score:.4f}")

print("\n TF-IDF RESULTS:")
for rank, (docid, score, idx) in enumerate(search_tfidf(query, 5), start=1):
    print(f"{rank}. {docid}  |  score={score:.4f}")

print("\n BM25 → TF-IDF RERANK RESULTS:")
for rank, (docid, score, idx) in enumerate(search_bm25_tfidf(query, 30, 5), start=1):
    print(f"{rank}. {docid}  |  score={score:.4f}")


Enter your search query: recent major incidents and breaking news

 BM25 RESULTS:
1. doc_1931  |  score=12.1144
2. doc_1802  |  score=9.2333
3. doc_1372  |  score=8.4565
4. doc_1043  |  score=7.9621
5. doc_2530  |  score=7.8124

 TF-IDF RESULTS:
1. doc_1931  |  score=0.1803
2. doc_1372  |  score=0.0883
3. doc_1953  |  score=0.0827
4. doc_1802  |  score=0.0791
5. doc_1625  |  score=0.0790

 BM25 → TF-IDF RERANK RESULTS:
1. doc_1931  |  score=0.1803
2. doc_1372  |  score=0.0883
3. doc_1953  |  score=0.0827
4. doc_1802  |  score=0.0791
5. doc_1625  |  score=0.0790
